In [4]:
from transformers import AutoTokenizer, pipeline
import pandas as pd

In [5]:
# load the data

news = pd.read_csv('..\..\data\output.csv')

In [7]:
news

,link,title,content
0,https://www.today.com/food/news/applebees-doll...,Applebee’s brings back fan-favorite menu item ...,Sections Shows More Follow today More Brands O...
1,https://www.today.com/popculture/tv/love-is-bl...,‘Love Is Blind’ star Taylor Rue is 'still tryi...,Sections Shows More Follow today More Brands O...
2,https://www.today.com/health/womens-health/ob-...,OB-GYNs share 7 things they never do when it c...,Sections Shows More Follow today More Brands T...
3,https://www.today.com/popculture/jodie-turner-...,Jodie Turner-Smith files for divorce from Josh...,Sections Shows More Follow today More Brands J...
4,https://www.today.com/parents/how-old-too-old-...,How old is too old to trick or treat?,Sections Shows More Follow today More Brands H...
...,...,...,...
1185,https://www.today.com/news/onewheel-electric-s...,Onewheel electric skateboards recalled after 4...,Sections Shows More Follow today More Brands H...
1186,https://www.today.com/popculture/music/beyonce...,Tina Knowles praises Beyoncé and Taylor Swift ...,Sections Shows More Follow today More Brands T...
1187,https://www.today.com/news/powerball-no-winner...,"Powerball draw produces no winners, pushing ja...",Sections Shows More Follow today More Brands S...
1188,https://www.today.com/popculture/travis-kelce-...,Travis Kelce and Chiefs hilariously trade Swif...,Sections Shows More Follow today More Brands A...


In [67]:
news_short = news.head(100)

In [46]:
def get_sentiment(text):
    encoded_input = tokenizer(
        text, 
        truncation=True, 
        max_length=MAX_TOKENS, 
        return_tensors=None
    )
    truncated_text = tokenizer.decode(encoded_input["input_ids"])
    sentiment_check = pipeline(
        model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
        top_k=3
    )
    sentiment_result = sentiment_check(truncated_text)

    # Extract the first label and score from the first list
    sentiment_label = sentiment_result[0][0]['label']
    sentiment_score = sentiment_result[0][0]['score']
    return sentiment_label, sentiment_score

In [68]:
# Create a temporary DataFrame to store the sentiment labels and scores
temp_df = news_short['content'].apply(lambda x: pd.Series(get_sentiment(x)))
temp_df.columns = ['sentiment', 'score']

# Use .loc to merge the temporary DataFrame with the original DataFrame
news_short = news_short.assign(sentiment=temp_df['sentiment'], score=temp_df['score'])

# Filter only positive sentiments
positive_news = news_short[news_short['sentiment'] == 'positive']

In [69]:
positive_news['link'].size

61

In [70]:
positive_news.sort_values(by=['score'], ascending=False)

,link,title,content,sentiment,score
66,https://www.today.com/food/elizabeth-heiskell-...,Elizabeth Heiskell is serving up nostalgia wit...,Sections Shows More Follow today More Brands T...,positive,0.858477
94,https://www.today.com/food/halloween-cocktails...,42 haunting Halloween cocktails for a boo-zy n...,Sections Shows More Follow today More Brands I...,positive,0.837433
52,https://www.today.com/shop/amazon-holiday-beau...,Amazon’s Holiday Beauty Haul has deals on Tatc...,Sections Shows More Follow today More Brands ...,positive,0.814940
54,https://www.today.com/shop/best-amazon-legging...,Amazon reviewers can’t stop talking about thes...,Sections Shows More Follow today More Brands T...,positive,0.793598
98,https://www.today.com/shop/toy-insider-hot-toy...,The Toy Insider says these holiday toys are ‘e...,Sections Shows More Follow today More Brands B...,positive,0.781718
...,...,...,...,...,...
86,https://www.today.com/popculture/movies/martin...,Martin Scorsese releases a list of 59 must-wat...,Sections Shows More Follow today More Brands M...,positive,0.427322
81,https://www.today.com/popculture/tv/how-to-str...,Here's how to watch 'Friends' on streaming ser...,Sections Shows More Follow today More Brands A...,positive,0.422681
82,https://www.today.com/shop/robecurls-original-...,This viral curling trend isn't going anywhere ...,Sections Shows More Follow today More Brands P...,positive,0.414063
69,https://www.today.com/parents/family/is-daycar...,Dear new parent: Daycare won’t screw up your b...,Sections Shows More Follow today More Brands I...,positive,0.404506
